**Get the metrices for DISP18 and DISP50**

In [6]:
import numpy as np
import matplotlib.image as im
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
import cv2
from os import listdir

In [105]:
from evaluation_utils import compute_errors, convert_disps_to_depths_kitti, width_to_focal

In [1]:
!unzip -qq /content/drive/MyDrive/ForMetrics/MetricsData.zip

In [106]:
TEST_TRN = "/content/MetricsData/TestImages/2011_09_26_drive_0001_sync"
TEST_VAL = "/content/MetricsData/TestImages/2011_09_26_drive_0020_sync"
GNDT_TRN = "/content/MetricsData/GroundTruth/train/2011_09_26_drive_0001_sync"
GNDT_VAL = "/content/MetricsData/GroundTruth/val/2011_09_26_drive_0020_sync"
DISP18 = "/content/MetricsData/Disparities/Resnet18"
DISP50 = "/content/MetricsData/Disparities/Resnet50"

In [107]:
def load_gt_disp_KITTI(paths):
    gt_disparities = []
    for path in paths:
        for file1 in sorted(listdir(path)):
            fpath = Path(path) / file1
            print(fpath)
            disp = cv2.imread(str(fpath), -1)
            disp = disp.astype(np.float32) / 256
            gt_disparities.append(disp)            
    return gt_disparities

In [108]:
gt_disparities = load_gt_disp_KITTI([GNDT_VAL, GNDT_TRN])

/content/MetricsData/GroundTruth/val/2011_09_26_drive_0020_sync/0000000070.png
/content/MetricsData/GroundTruth/val/2011_09_26_drive_0020_sync/0000000071.png
/content/MetricsData/GroundTruth/val/2011_09_26_drive_0020_sync/0000000072.png
/content/MetricsData/GroundTruth/val/2011_09_26_drive_0020_sync/0000000073.png
/content/MetricsData/GroundTruth/val/2011_09_26_drive_0020_sync/0000000074.png
/content/MetricsData/GroundTruth/val/2011_09_26_drive_0020_sync/0000000075.png
/content/MetricsData/GroundTruth/val/2011_09_26_drive_0020_sync/0000000076.png
/content/MetricsData/GroundTruth/val/2011_09_26_drive_0020_sync/0000000077.png
/content/MetricsData/GroundTruth/val/2011_09_26_drive_0020_sync/0000000078.png
/content/MetricsData/GroundTruth/val/2011_09_26_drive_0020_sync/0000000079.png
/content/MetricsData/GroundTruth/train/2011_09_26_drive_0001_sync/0000000090.png
/content/MetricsData/GroundTruth/train/2011_09_26_drive_0001_sync/0000000091.png
/content/MetricsData/GroundTruth/train/2011_09_2

In [109]:
len(gt_disparities), gt_disparities[0].shape

(20, (375, 1242))

In [110]:
#pred_disparities = np.load(args.predicted_disp_path)
pred_disparities50 = np.load(Path(DISP50) / "disparities_pp.npy")

In [111]:
pred_disparities50.shape

(20, 256, 512)

In [112]:
gt_depths, pred_depths, pred_disparities_resized = convert_disps_to_depths_kitti(gt_disparities, pred_disparities50)

In [113]:
len(gt_depths), gt_depths[0].shape, len(pred_depths) ,pred_depths[0].shape, len(pred_disparities_resized), pred_disparities_resized[0].shape

(20, (375, 1242), 20, (375, 1242), 20, (375, 1242))

In [114]:
num_samples = len(gt_depths)
rms     = np.zeros(num_samples, np.float32)
log_rms = np.zeros(num_samples, np.float32)
abs_rel = np.zeros(num_samples, np.float32)
sq_rel  = np.zeros(num_samples, np.float32)
d1_all  = np.zeros(num_samples, np.float32)
a1      = np.zeros(num_samples, np.float32)
a2      = np.zeros(num_samples, np.float32)
a3      = np.zeros(num_samples, np.float32)

In [115]:
#width = 1242
class args:    #Jag: Asumption Do some research
  min_depth = np.min(gt_depths)
  max_depth = np.max(gt_depths) #Jag:
  split = 'kitti'

In [116]:
args.min_depth, args.max_depth, args.split, np.max(gt_depths)

(4.790153755366662, 389.630358, 'kitti', 389.630358)

In [117]:
for i in range(num_samples):
    
    gt_depth = gt_depths[i]
    pred_depth = pred_depths[i]

    pred_depth[pred_depth < args.min_depth] = args.min_depth
    pred_depth[pred_depth > args.max_depth] = args.max_depth
    
    if args.split == 'kitti':
        gt_disp = gt_disparities[i]
        mask = gt_disp > 0
        pred_disp = pred_disparities_resized[i]

        disp_diff = np.abs(gt_disp[mask] - pred_disp[mask])
        bad_pixels = np.logical_and(disp_diff >= 3, (disp_diff / gt_disp[mask]) >= 0.05)
        d1_all[i] = 100.0 * bad_pixels.sum() / mask.sum()

    abs_rel[i], sq_rel[i], rms[i], log_rms[i], a1[i], a2[i], a3[i] = compute_errors(gt_depth[mask], pred_depth[mask])

print("{:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}".format('abs_rel', 'sq_rel', 'rms', 'log_rms', 'd1_all', 'a1', 'a2', 'a3'))
print("{:10.4f}, {:10.4f}, {:10.3f}, {:10.3f}, {:10.3f}, {:10.3f}, {:10.3f}, {:10.3f}".format(abs_rel.mean(), sq_rel.mean(), rms.mean(), log_rms.mean(), d1_all.mean(), a1.mean(), a2.mean(), a3.mean()))

   abs_rel,     sq_rel,        rms,    log_rms,     d1_all,         a1,         a2,         a3
    0.8121,    26.3189,     34.159,      1.936,    100.000,      0.012,      0.027,      0.045
